### Javier Mombiela
### Carnet: 20067

#### Laboratorio #5 – Threat hunting

#### Parte 1 – Filtrado y preprocesamiento

In [93]:
import pandas as pd
from clasificador import *

2. Cargar la informacion de large_even.json

In [94]:
# Cargar el archivo large_even.json en una lista
with open('large_even.json', 'r') as file:
    data = [json.loads(line) for line in file]

# Mostrar la cantidad total de registros
total_registros = len(data)
print(f"Cantidad de registros: {total_registros}")

Cantidad de registros: 746909


3. y 4. Cargar solamente DNS y mostrar cantidad

In [95]:
# Filtrar únicamente los registros DNS
registros_dns = [registro for registro in data if registro['event_type'] == 'dns']

# Mostrar la nueva cantidad de registros filtrados
cantidad_registros_dns = len(registros_dns)
print(f"Cantidad de registros DNS: {cantidad_registros_dns}")

Cantidad de registros DNS: 15749


5. Mostrar 2 registros

In [96]:
for i in range(2):
    print("Registro", i+1, ":")
    print(registros_dns[i])
    print()

Registro 1 :
{'timestamp': '2017-07-22T17:33:16.661646-0500', 'flow_id': 1327836194150542, 'pcap_cnt': 22269, 'event_type': 'dns', 'vlan': 110, 'src_ip': '2001:0dbb:0c18:0011:0260:6eff:fe30:0863', 'src_port': 59680, 'dest_ip': '2001:0500:0001:0000:0000:0000:803f:0235', 'dest_port': 53, 'proto': 'UDP', 'dns': {'type': 'query', 'id': 15529, 'rrname': 'api.wunderground.com', 'rrtype': 'A', 'tx_id': 0}}

Registro 2 :
{'timestamp': '2017-07-22T17:33:24.990320-0500', 'flow_id': 2022925111925872, 'pcap_cnt': 54352, 'event_type': 'dns', 'vlan': 110, 'src_ip': '2001:0dbb:0c18:0011:0260:6eff:fe30:0863', 'src_port': 38051, 'dest_ip': '2001:0500:0003:0000:0000:0000:0000:0042', 'dest_port': 53, 'proto': 'UDP', 'dns': {'type': 'query', 'id': 58278, 'rrname': 'stork79.dropbox.com', 'rrtype': 'A', 'tx_id': 0}}



6. Shape del DNS dataframe

In [97]:
# Normalizar la lista de registros DNS y convertirla en un DataFrame
df_dns = pd.json_normalize(registros_dns)

# Mostrar el shape del DataFrame
print(f"Shape del DataFrame: {df_dns.shape}")
df_dns.head()

Shape del DataFrame: (15749, 18)


,timestamp,flow_id,pcap_cnt,event_type,vlan,src_ip,src_port,dest_ip,dest_port,proto,dns.type,dns.id,dns.rrname,dns.rrtype,dns.tx_id,dns.rcode,dns.ttl,dns.rdata
0,2017-07-22T17:33:16.661646-0500,1327836194150542,22269,dns,110,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,59680,2001:0500:0001:0000:0000:0000:803f:0235,53,UDP,query,15529,api.wunderground.com,A,0.0,NaN,NaN,NaN
1,2017-07-22T17:33:24.990320-0500,2022925111925872,54352,dns,110,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,38051,2001:0500:0003:0000:0000:0000:0000:0042,53,UDP,query,58278,stork79.dropbox.com,A,0.0,NaN,NaN,NaN
2,2017-07-22T17:33:27.379891-0500,578544790391795,54519,dns,150,192.168.205.170,31393,192.168.207.4,53,UDP,query,54724,hpca-tier2.office.aol.com.ad.aol.aoltw.net,A,0.0,NaN,NaN,NaN
3,2017-07-22T17:33:27.380146-0500,578544790391795,54520,dns,150,192.168.207.4,53,192.168.205.170,31393,UDP,answer,54724,hpca-tier2.office.aol.com.ad.aol.aoltw.net,NaN,NaN,NXDOMAIN,NaN,NaN
4,2017-07-22T17:33:27.380146-0500,578544790391795,54520,dns,150,192.168.207.4,53,192.168.205.170,31393,UDP,answer,54724,<root>,SOA,NaN,NXDOMAIN,20864.0,NaN


7. Filtrar por DNS Tipo A

In [98]:
# Filtrar DNS tipo A
registros_dns_A = df_dns[df_dns['dns.rrtype'] == 'A']

# Mostrar la cantidad de registros DNS tipo A
cantidad_registros_dns_A = len(registros_dns_A)
print(f"Cantidad de registros DNS tipo A: {cantidad_registros_dns_A}")

Cantidad de registros DNS tipo A: 2849


8. Filtrar dominios unicos

In [99]:
# Filtrar los dominios únicos
dominios_unicos = registros_dns_A['dns.rrname'].unique()

# Mostrar la cantidad de dominios únicos
cantidad_dominios_unicos = len(dominios_unicos)
print(f"Cantidad de dominios únicos: {cantidad_dominios_unicos}")

Cantidad de dominios únicos: 177


9. Obtener TLD de dominios

Prompt utilizado: "Hola me podrias ayudar a escribir una funcion en python para obtener el TLD de un dominio por favor. Aca te dejo unos ejemplos para que te puedas guiar: Por ejemplo, para api.wunderground.com el TLD es wunderground.com, para safebrowsing.clients.google.com.home, el TLD es home"

In [100]:
import tldextract

def obtener_tld(dominio):
    # Extrae el TLD utilizando tldextract
    resultado = tldextract.extract(dominio)
    tld = resultado.domain
    
    if resultado.suffix:
        tld += '.' + resultado.suffix

    return tld

In [101]:
# Ejemplos de uso
print(obtener_tld("api.wunderground.com"))  
print(obtener_tld("safebrowsing.clients.google.com.home")) 

wunderground.com
home


10. Obtener el TLD de dominios unicos

In [102]:
tlds = [obtener_tld(dominio) for dominio in dominios_unicos]

tlds_df = pd.DataFrame(tlds, columns=['domain_tld'])
tlds_df.head(10)

,domain_tld
0,wunderground.com
1,dropbox.com
2,aoltw.net
3,home
4,mozilla.com
5,metasploit.com
6,aol.com
7,aoltw.net
8,aol.com
9,aol.com


#### Parte 2 – Data Science

11. Utilizar el clasificador para asignar el resultado a un nuevo dataframe

In [103]:
df = clasificacion(tlds_df)
df.head()

c:\Users\rjmom\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:299: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.0.2 when using version 1.2.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


,domain_tld,isDGA
0,wunderground.com,0
1,dropbox.com,1
2,aoltw.net,1
3,home,0
4,mozilla.com,0


12. Filtrar DGAs == 1

In [104]:
# Filtrar los dominios considerados como DGA
df_dga = df[df['isDGA'] == 1]

# Mostrar los dominios DGA
print("Dominios considerados como DGA:")
print(df_dga)


Dominios considerados como DGA:
               domain_tld  isDGA
1             dropbox.com      1
2               aoltw.net      1
5          metasploit.com      1
7               aoltw.net      1
10              aoltw.net      1
15              aoltw.net      1
18          110phpmyadmin      1
25            windows.com      1
27          ntkrnlpa.info      1
32              aoltw.net      1
34         sql-ledger.org      1
35    backtrack-linux.org      1
40              aoltw.net      1
41              aoltw.net      1
42              aoltw.net      1
45              aoltw.net      1
47              aoltw.net      1
48         phpmyadmin.net      1
51          microsoft.com      1
53              aoltw.net      1
56              aoltw.net      1
60         postgresql.org      1
62            freepbx.org      1
64      bigflickrfeed.com      1
72            freepbx.org      1
75          microsoft.com      1
77          microsoft.com      1
78             flickr.com      1
80         

In [105]:
# Eliminar duplicados y obtener los registros únicos
df_dga_unicos = df_dga.drop_duplicates(subset=['domain_tld'])

# Mostrar la cantidad de registros únicos
cantidad_registros_unicos_dga = df_dga_unicos.shape[0]
print("\nCantidad de registros únicos de dominios DGA:", cantidad_registros_unicos_dga)


Cantidad de registros únicos de dominios DGA: 35


#### Parte 3 – Dominio experto

13. Funcion para revisar TLD

Prompt utilizado: "Hola utilizando el archivo .csv 'top-1m.csv' me podrias ayudar a crear una funcion en python que utilice la lista de dicho archivo y devuelva 0 si el TLD de mi dataframe df_dga_unicos se encuentra en la lista y 1 si no esta por favor" (le pego el formato del top-1m.csv y del head de mi dataframe)

In [106]:
def dominio_experto(tld):
    # Cargar la lista de un millón de TLDs
    df_top_1m = pd.read_csv('top-1m.csv', header=None, names=['ranking', 'domain'])

    # Extraer los TLDs de la lista
    tlds_lista = set(df_top_1m['domain'])

    if tld in tlds_lista:
        return 0
    else:
        return 1

In [107]:
# Ejemplo de uso
resultado = dominio_experto('google.com')
print(resultado)  # Output: 0 (Google.com está en la lista)

resultado = dominio_experto('money1.com')
print(resultado)  # Output: 1 (money1.com no está en la lista)

0
1


14. Determinar si los DGA unicas estan en la lista

In [108]:
# Crear una nueva columna en df_dga_unicos para almacenar los resultados de la función dominio_experto
df_dga_unicos['sospechoso'] = df_dga_unicos['domain_tld'].apply(dominio_experto)

df_dga_unicos.head()

C:\Users\rjmom\AppData\Local\Temp\ipykernel_14776\969752244.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dga_unicos['sospechoso'] = df_dga_unicos['domain_tld'].apply(dominio_experto)


,domain_tld,isDGA,sospechoso
1,dropbox.com,1,0
2,aoltw.net,1,1
5,metasploit.com,1,0
18,110phpmyadmin,1,1
25,windows.com,1,0


In [109]:
# Filtrar aquellos dominios que se encuentran en la lista proporcionada
dgas_sospechosos = df_dga_unicos[df_dga_unicos['sospechoso'] == 1]

# Eliminar duplicados y obtener los registros únicos
dgas_sospechosos_unicos = dgas_sospechosos.drop_duplicates(subset=['domain_tld'])

# Mostrar la cantidad de registros únicos
cantidad_registros_unicos_en_lista = dgas_sospechosos_unicos.shape[0]
print(f"Cantidad de registros únicos de dominios DGA que se encuentran en la lista: {cantidad_registros_unicos_en_lista}")

Cantidad de registros únicos de dominios DGA que se encuentran en la lista: 13


In [110]:
print(dgas_sospechosos_unicos[['domain_tld', 'sospechoso']])

               domain_tld  sospechoso
2               aoltw.net           1
18          110phpmyadmin           1
27          ntkrnlpa.info           1
34         sql-ledger.org           1
35    backtrack-linux.org           1
64      bigflickrfeed.com           1
82        malwarecity.com           1
89            cakephp.org           1
97              ecvps.com           1
160  vtlfccmfxlkgifuf.com           1
162        macromates.com           1
167  ejfodfmfxlkgifuf.xyz           1
169        192.168.22.201           1


15. Funcion para buscar fecha de creacion del TLD

Prompt utilizado: "Hola, dados los siguientes dgas sospechosos, me podrias ayudar a crear una funcion que en base al TLD, me devuelva la fecha de creacion de dicho dga por favor." (le pego la lista de los 13 dgas)

In [111]:
import pythonwhois

def obtener_fecha_creacion(dominio):
    try:
        detalles = pythonwhois.get_whois(dominio)
        fecha_creacion = detalles['creation_date']
        if fecha_creacion:
            # Formatear la fecha a un formato más legible
            fecha_formateada = fecha_creacion[0].strftime('%d-%m-%Y %H:%M:%S')
            return fecha_formateada
        else:
            return None
    except Exception as e:
        # print(f"No se pudo obtener la información de {dominio}: {e}")
        return None

In [112]:
# Ejemplos de uso
print(obtener_fecha_creacion("google.com"))
print(obtener_fecha_creacion("aoltw.net"))

15-09-1997 04:00:00
10-01-2000 04:09:19


16. Mostrar la fecha de creacion de los 13 dominios finales

In [113]:
# mostrar la fecha de creación de los dominios sospechosos
dgas_sospechosos_unicos['fecha_creacion'] = dgas_sospechosos_unicos['domain_tld'].apply(obtener_fecha_creacion)

print(dgas_sospechosos_unicos[['domain_tld', 'sospechoso', 'fecha_creacion']])

               domain_tld  sospechoso       fecha_creacion
2               aoltw.net           1  10-01-2000 04:09:19
18          110phpmyadmin           1                 None
27          ntkrnlpa.info           1  27-06-2023 08:18:55
34         sql-ledger.org           1  08-09-2000 16:25:09
35    backtrack-linux.org           1  29-04-2009 00:26:42
64      bigflickrfeed.com           1                 None
82        malwarecity.com           1  06-02-2008 15:42:35
89            cakephp.org           1  13-06-2005 23:07:32
97              ecvps.com           1  21-05-2009 14:24:02
160  vtlfccmfxlkgifuf.com           1                 None
162        macromates.com           1  05-02-2003 00:25:14
167  ejfodfmfxlkgifuf.xyz           1                 None
169        192.168.22.201           1                 None


Este listado final, el cual incluye los ultimos 13 dominios sospechocos con sus fechas, nos puede ayudar bastante a determinar cuales dominios si son sospechosos. Al ver la fecha, sabemos que una fecha antigua significa que si es una empresa reconocida y legitima, por lo que podemos descartar dichos dominios como sospechosos. Los dominios que tienen fecha reciente o no tienen fechas, pueden ser categorizados como sospechosos y estos incluyen los siguientes dominios:

- 110phpmyadmin
- ntkrnlpa.info
- bigflickrfeed.com
- vtlfccmfxlkgifuf.com
- ejfodfmfxlkgifuf.xy
- 192.168.22.201

17. Dominios con patrones aleatorios

En el inciso anterior logramos descartar 7 dominios como sospechosos, lo que nos deja con 6 sospechosos, los cuales estan listados arriba. Al ver el nombre de estos 6 sospechosos podemos ver que hay 2 que sobresalen, ya que son patrones aleatorios sin ningun sentido y por lo tanto podemos identificar que estos son los 2 dominions sospechosos del dataset.

- vtlfccmfxlkgifuf.com
- ejfodfmfxlkgifuf.xy